In [2]:
import keras_tuner
import pickle
from sklearn.utils import class_weight
from model_definition import *
from tuner_trial_functions import *
from preprocessing import vocab_size

In [3]:
with open('./data/train_preprocessed_routes', 'rb') as f:
    X_train, y_train = pickle.load(f).values()
#with open('./data/test_preprocessed_routes', 'rb') as f:
#    X_test, y_test = pickle.load(f).values()
with open('./data/val_preprocessed_routes', 'rb') as f:
    X_val, y_val = pickle.load(f).values()

In [4]:
X_train[:, 0] = 3
#X_test[:, 0] = 3
X_val[:, 0] = 3

In [5]:
tuner_num = 0 #initialization
hp_type = ""

In [6]:
# Variables:

#constant
num_classes = 9
epochs = 40

# Iteration one, 68 trials :

# num_layers   =  2    -  8,   step 2   ->   2
# d_model      =  64   -  512, step 64  ->   64
# dff          =  512  -  2048, step 256 ->  768
# num_heads    =  4    -  10,   step 2   ->  10
# dropout_rate =  0.1  -  0.4,  step 0.1 ->  0.1
# warmup_steps =  2500 -  5500, step 500 ->  3500



# Iteration two, 100 trials:

# num_layers   =  1    -  4,   step 1   ->  3
# d_model      =  16   -  128, step 8  -> 128
# dff          =  512  -  1024, step 32 -> 736
# num_heads    =  8    -  14,   step 1   -> 8
# dropout_rate =  0.04 -  0.24,  step 0.02 -> 0.2
# warmup_steps =  2500 -  4750, step 250 -> 3250
# beta_1       =  0.79 -  0.95, step 0.02 -> 0.79
# beta_2       =  0.95 -  0.99, step 0.005 -> 0.98
# epsilon      = 1e-11 -  1e-7, step NA   -> 6.35e-08

# Iteration three, 99 trials:

# num_layers   =  1    -  4,   step 1   ->  
    # default: 2
# d_model      =  16   -  192, step 16  -> 
    # default: 128
# dff          =  512  -  1280, step 64 ->
    # default: 768
# num_heads    =  8    -  14,   step 1   -> 
    # default: 10
# dropout_rate =  0.125 -  0.30,  step 0.025 -> 
    # default: 0.2
# warmup_steps =  2000 -  7000, step 500 -> 
    # default: 4000
# beta_1       =  0.74 -  0.93, step 0.0025 -> 
    # default: 0.8
# beta_2       =  0.95 -  0.99, step 0.005 -> 
    # default: 0.98
# epsilon      = 1e-11 -  1e-7, sample 'log' ->
    # default: 1e-8
# global_batch =  16   -  128,  step 16  ->
    # default: 64


# Implement:
# global_batch =  16   -  128,  step 16  ->

# learning_rate = CustomSchedule(d_model, warmup_steps=warmup_steps)
# optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [7]:
class MyHyperModel(keras_tuner.HyperModel):
    def build(self, hp):     
        num_classes = 9
        self.batch_size = hp.get('batch_size')
            
        strategy = tf.distribute.MirroredStrategy()
        print("Number of devices: {}".format(strategy.num_replicas_in_sync))

        #Open a strategy scope.
        with strategy.scope():
        #Everything that creates variables should be under the strategy scope.
        #In general this is only model construction & `compile()`.
            model = EncoderClassifier(
                num_layers=hp.get('num_layers'),
                d_model=hp.get('d_model'),
                num_heads=hp.get('num_heads'),
                dff=hp.get('dff'),
                vocab_size=vocab_size,
                num_classes=num_classes,
                dropout_rate=hp.get('dropout_rate'),
                activation=hp.get('activation'),
                sequential=hp.get('sequential')
            )

            learning_rate = CustomSchedule(hp.get('d_model'), warmup_steps=hp.get('warmup_steps'))
            optimizer = tf.keras.optimizers.Adam(learning_rate, 
                                    beta_1=hp.get("beta_1"),
                                    beta_2=hp.get("beta_2"),
                                    epsilon=hp.get("epsilon"))
            
            model.compile(
                loss=tf.keras.losses.SparseCategoricalCrossentropy(),
                optimizer=optimizer,
                metrics=['accuracy'])    
        
        return model

    def fit(self, hp, model, x, y, epochs, validation_data, verbose=1, **kwargs):
        batch_size = self.batch_size
        steps_per_epoch = np.ceil((np.shape(x)[0]/batch_size))
        
        # Convert the datasets to tf.data.Dataset.        
        train_dist = tf.data.Dataset.from_tensor_slices((X_train, y_train)).repeat(epochs).batch(batch_size)
        val_dist = tf.data.Dataset.from_tensor_slices((X_val, y_val)).repeat(epochs).batch(batch_size)

        history = model.fit(train_dist, epochs=epochs, validation_data=val_dist, steps_per_epoch=steps_per_epoch, 
                  validation_steps=1, verbose=1)
        
        return history

In [12]:
def create_base_hp(hp=keras_tuner.HyperParameters()):  #, hp_type=hp_type):  
    #hp.Fixed("Tuning_Mode", value=hp_type)
    
    # These Hyperparameters affect the size/complexity of the model layers
    num_layers = hp.Int("num_layers", min_value=1, max_value=4, default=2, step=1)
    d_model = hp.Int("d_model", min_value=16, max_value=192, default=128, step=16)
    dff = hp.Int("dff", min_value=512, max_value=1280, default=768, step=64)
    num_heads = hp.Int("num_heads", min_value=8, max_value=14, default=10, step=1)
    dropout_rate = hp.Float("dropout_rate", min_value=0.125, max_value=0.3, default=0.2, step=0.025)
        
    #if(hp_type == "learn" or hp_type == "base"):
    # These Hyperparameters affect the learning rate
    warmup_steps = hp.Int("warmup_steps", min_value=2000, max_value=7000, default=4000, step=500)
    batch_size = hp.Int("batch_size", min_value=128, max_value=256, step=16)
    beta_1 = hp.Float("beta_1", min_value=0.74, max_value=0.93, default= .8, step=0.0025)
    beta_2 = hp.Float("beta_2", min_value=0.95, max_value=0.99, default= .98, step=0.005)
    epsilon = hp.Float("epsilon", min_value=1e-9, max_value=5e-7, default= 1e-8, sampling="log")

    #if(hp_type == "arch" or hp_type == "base"):
    hp.Choice("activation", values=['relu', 'swish'], default='relu')
    hp.Boolean("sequential", default=True)
        
    return hp

In [13]:
def get_tuner(tune_new=True, trials=10, tuner_num=tuner_num):#, hp_type=hp_type):
    #hp = create_updated_hp(tuner_num=tuner_num, hp_type=hp_type)
    hp = create_base_hp()
    return keras_tuner.BayesianOptimization(
        hypermodel = MyHyperModel(),
        hyperparameters = hp,
        #tune_new_entries = tune_new,
        objective="val_accuracy",
        max_trials=trials,
        overwrite=False,
        directory="tuners",
        project_name=f'tuner{tuner_num}',
    )

In [14]:
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(y_train),
                                                  y=y_train) 
class_weights=dict(zip(np.unique(y_train), class_weights))

my_callbacks = tf.keras.callbacks.EarlyStopping(
    monitor="val_accuracy", 
    min_delta=0.05, patience=3,
    verbose=2, baseline=0.40, start_from_epoch=1)

In [15]:
base_tuner = get_tuner(trials=99, tuner_num=5)
   
base_tuner.search(x=X_train, y=y_train, epochs=50, validation_data=(X_val, y_val))

Trial 11 Complete [00h 00m 30s]

Best val_accuracy So Far: 0.5052083134651184
Total elapsed time: 00h 25m 46s

Search: Running Trial #12

Value             |Best Value So Far |Hyperparameter
1                 |4                 |num_layers
128               |96                |d_model
896               |512               |dff
11                |10                |num_heads
0.2               |0.275             |dropout_rate
4000              |3000              |warmup_steps
208               |192               |batch_size
0.875             |0.7825            |beta_1
0.965             |0.95              |beta_2
8.024e-08         |4.5878e-08        |epsilon
swish             |swish             |activation
False             |False             |sequential

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:G

2023-03-28 19:09:30.933045: W tensorflow/core/grappler/optimizers/data/auto_shard.cc:784] AUTO sharding policy will apply DATA sharding policy as it failed to apply FILE sharding policy because of the following reason: Found an unshardable source dataset: name: "TensorSliceDataset/_2"
op: "TensorSliceDataset"
input: "Placeholder/_0"
input: "Placeholder/_1"
attr {
  key: "Toutput_types"
  value {
    list {
      type: DT_INT64
      type: DT_INT64
    }
  }
}
attr {
  key: "_cardinality"
  value {
    i: 19076
  }
}
attr {
  key: "is_files"
  value {
    b: false
  }
}
attr {
  key: "metadata"
  value {
    s: "\n\027TensorSliceDataset:7101"
  }
}
attr {
  key: "output_shapes"
  value {
    list {
      shape {
        dim {
          size: 22
        }
      }
      shape {
      }
    }
  }
}
attr {
  key: "replicate_on_split"
  value {
    b: false
  }
}
experimental_type {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_DATASET
    args {
      type_id: TFT_PRODUCT
      args {
  

Epoch 1/50


Traceback (most recent call last):
  File "/home/milleraa/.local/lib/python3.8/site-packages/keras_tuner/engine/base_tuner.py", line 266, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/milleraa/.local/lib/python3.8/site-packages/keras_tuner/engine/base_tuner.py", line 231, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/milleraa/.local/lib/python3.8/site-packages/keras_tuner/engine/tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/home/milleraa/.local/lib/python3.8/site-packages/keras_tuner/engine/tuner.py", line 214, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/tmp/ipykernel_3346363/3542397656.py", line 46, in fit
    history = model.fit(train_dist, epochs=epochs, validation_data=val_dist, steps_per_epoch=steps_per_epoch,
  File "/home/milleraa/.local/lib/python

RuntimeError: Number of consecutive failures excceeded the limit of 3.
Traceback (most recent call last):
  File "/home/milleraa/.local/lib/python3.8/site-packages/keras_tuner/engine/base_tuner.py", line 266, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/milleraa/.local/lib/python3.8/site-packages/keras_tuner/engine/base_tuner.py", line 231, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/milleraa/.local/lib/python3.8/site-packages/keras_tuner/engine/tuner.py", line 287, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/home/milleraa/.local/lib/python3.8/site-packages/keras_tuner/engine/tuner.py", line 214, in _build_and_fit_model
    results = self.hypermodel.fit(hp, model, *args, **kwargs)
  File "/tmp/ipykernel_3346363/3542397656.py", line 46, in fit
    history = model.fit(train_dist, epochs=epochs, validation_data=val_dist, st

In [ ]:
tuner_num = 4
hp_type = ""

base_num = 10
layer_num = 6
learn_num = 6
arch_num = 3

iterations = 1
num_trials = iterations*(layer_num+learn_num+arch_num)+base_num

assert base_num >= 10
print(f'Number of trials: {num_trials}')

In [12]:
#base_hp = create_hp(hp_type="base")
#layer_hp = create_hp(hp_type="layer")
#learn_hp = create_hp(hp_type="learn")
#arch_hp = create_hp(hp_type="arch")

In [15]:
#base_tuner = get_tuner(create_hp(hp_type="base"), trials=base_num, tuner_num=tuner_num)

# run 10 models to get a starting point
#base_tuner.search(x=X_train, y=y_train, epochs=3, validation_data=(X_val, y_val), 
             #class_weight=class_weights)

# run 4 * 15 models: 6 focused on tuning layer values, 6 focused on tuning learn values, 
# and 3 focused on changing model architecture
for i in range(1, iterations+1):
    layer_trial = layer_num * i + base_num
    learn_trial = learn_num * i + base_num + layer_trial
    arch_trial  = arch_num  * i + base_num + learn_trial
    
    
    layer_tuner = get_tuner(tune_new=False, trials=layer_trial, tuner_num=tuner_num, hp_type='layer')
    layer_tuner.search(x=X_train, y=y_train, epochs=3, validation_data=(X_val, y_val), 
             class_weight=class_weights)
    
    
    learn_tuner = get_tuner(tune_new=False, trials=learn_trial, tuner_num=tuner_num, hp_type='learn')
    learn_tuner.search(x=X_train, y=y_train, epochs=3, validation_data=(X_val, y_val), 
             class_weight=class_weights)
    
    arch_tuner = get_tuner(tune_new=False, trials=arch_trial, tuner_num=tuner_num, hp_type='arch')
    arch_tuner.search(x=X_train, y=y_train, epochs=3, validation_data=(X_val, y_val), 
             class_weight=class_weights)

INFO:tensorflow:Reloading Tuner from tuners/tuner4/tuner0.json

Search: Running Trial #11

Value             |Best Value So Far |Hyperparameter
base              |base              |Tuning_Mode
4                 |2                 |num_layers
144               |64                |d_model
640               |960               |dff
13                |9                 |num_heads
0.125             |0.275             |dropout_rate
2000              |3000              |warmup_steps
48                |32                |batch_size
0.9225            |0.84              |beta_1
0.98              |0.965             |beta_2
9.701e-09         |1.248e-08         |epsilon
swish             |swish             |activation
True              |False             |sequential

Epoch 1/3
398/398 [==============================] - 28s 46ms/step - loss: 1.7862 - accuracy: 0.2606 - val_loss: 1.5006 - val_accuracy: 0.4076
Epoch 2/3
263/398 [==================>...........] - ETA: 4s - loss: 1.4945 - accuracy: 0.36

KeyboardInterrupt: 

In [8]:
def create_updated_hp(hp=keras_tuner.HyperParameters(), hp_type=hp_type, tuner_num=tuner_num):
    trials_np = np.zeros(0)
    base_dir = f'tuners/tuner{tuner_num}'
    num_trials = get_num_trials(base_dir)
    trials_np = create_trials_np(num_trials, base_dir)

    tuner_df = create_tuner_df(num_trials, trials_np)
    df = best_trials(tuner_df, num_trials=1)    
    
    hp.Fixed("Tuning_Mode", value=hp_type)
           
    if (hp_type == "layer" or hp_type == "base"):
        # These Hyperparameters affect the size/complexity of the model layers
        num_layers = hp.Int("num_layers", min_value=1, max_value=4, default=2, step=1)
        d_model = hp.Int("d_model", min_value=16, max_value=192, default=128, step=16)
        dff = hp.Int("dff", min_value=512, max_value=1280, default=768, step=64)
        num_heads = hp.Int("num_heads", min_value=8, max_value=14, default=10, step=1)
        dropout_rate = hp.Float("dropout_rate", min_value=0.125, max_value=0.3, default=0.2, step=0.025)
        if(hp_type != "base"):
            hp.Fixed("warmup_steps", value=df['warmup_steps'].iloc[0])
            hp.Fixed("batch_size", value=df['batch_size'].iloc[0])
            hp.Fixed("beta_1", value=df['beta_1'].iloc[0])
            hp.Fixed("beta_2", value=df['beta_2'].iloc[0])
            hp.Fixed("epsilon", value=df['epsilon'].iloc[0])

            hp.Fixed("activation", value='relu')
            hp.Fixed("sequential", value=True)
    if (hp_type == "learn" or hp_type == "base"):
        # These Hyperparameters affect the learning rate
        if(hp_type != "base"):
            hp.Fixed("num_layers", value=df['num_layers'].iloc[0])
            hp.Fixed("d_model", value=df['d_model'].iloc[0])
            hp.Fixed("dff", value=df['dff'].iloc[0])
            hp.Fixed("num_heads", value=df['num_heads'].iloc[0])
            hp.Fixed("dropout_rate", value=df['dropout_rate'].iloc[0])
        
        warmup_steps = hp.Int("warmup_steps", min_value=2000, max_value=7000, default=4000, step=500)
        batch_size = hp.Int("batch_size", min_value=16, max_value=128, step=16)
        beta_1 = hp.Float("beta_1", min_value=0.74, max_value=0.93, default= .8, step=0.0025)
        beta_2 = hp.Float("beta_2", min_value=0.95, max_value=0.99, default= .98, step=0.005)
        epsilon = hp.Float("epsilon", min_value=1e-9, max_value=5e-7, default= 1e-8, sampling="log")
        
        if(hp_type != "base"):
            hp.Fixed("activation", value='relu')
            hp.Fixed("sequential", value=True)
    if(hp_type == "arch" or hp_type == "base"):
        if(hp_type != "base"):
            hp.Fixed("num_layers", value=df['num_layers'].iloc[0])
            hp.Fixed("d_model", value=df['d_model'].iloc[0])
            hp.Fixed("dff", value=df['dff'].iloc[0])
            hp.Fixed("num_heads", value=df['num_heads'].iloc[0])
            hp.Fixed("dropout_rate", value=df['dropout_rate'].iloc[0])
            
            hp.Fixed("warmup_steps", value=df['warmup_steps'].iloc[0])
            hp.Fixed("batch_size", value=df['batch_size'].iloc[0])
            hp.Fixed("beta_1", value=df['beta_1'].iloc[0])
            hp.Fixed("beta_2", value=df['beta_2'].iloc[0])
            hp.Fixed("epsilon", value=df['epsilon'].iloc[0])
        
        hp.Choice("activation", values=['relu', 'swish'], default='relu')
        hp.Boolean("sequential", default=True)

    return hp